In [1]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.3 MB/s eta 0:00:00


In [2]:
import subprocess
import os

# Install dependencies in Colab
if "google.colab" in str(get_ipython()):
    get_ipython().system('pip install openai-whisper ffmpeg torch redis scispacy spacy google-generativeai langchain langchain-google-genai langchain_community pyngrok fpdf python-docx')
    get_ipython().system('pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_md-0.5.3.tar.gz')
    get_ipython().system('apt-get install redis-server')

    # Start Redis server
    subprocess.Popen(['redis-server'], stdout=subprocess.PIPE)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 M

In [3]:
import torch
import whisper
import redis
import spacy
from pyngrok import ngrok
from tempfile import NamedTemporaryFile
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage

In [ ]:
# Load SciSpaCy Model
nlp = spacy.load("en_core_sci_md")

# Load Whisper Model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("base", device=device)

# Set up API key
os.environ["GOOGLE_API_KEY"] = "your_google_api_key"
llm = ChatGoogleGenerativeAI(model="gemini-pro")

/usr/local/lib/python3.11/dist-packages/spacy/language.py:2141: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 156MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializatio

In [ ]:
# Write the Streamlit app to app.py
%%writefile app.py
import streamlit as st
import torch
import whisper
import spacy
import os
import datetime
from tempfile import NamedTemporaryFile
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage
from fpdf import FPDF
from docx import Document

# Load SciSpaCy Model
nlp = spacy.load("en_core_sci_md")

# Load Whisper Model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("base", device=device)

# Set up API key
os.environ["GOOGLE_API_KEY"] = "your_google_api_key"
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Persist patient database across interactions
if "patient_db" not in st.session_state:
    st.session_state.patient_db = {}

patient_db = st.session_state.patient_db

st.title("🩺 AI-Powered Medical Documentation System")
st.write("Upload an audio file of a patient consultation to generate a structured medical report.")

st.subheader("Patient Information")

pid = st.text_input("Enter Patient ID (if existing) or leave blank to register a new patient")

if pid:
    if pid in patient_db:
        st.success(f"Patient {pid} found!")
        patient = patient_db[pid]
        st.write(f"**Name:** {patient['name']}")
        st.write(f"**Age/Sex:** {patient['age']}/{patient['sex']}")
    else:
        st.warning("Patient ID not found. Please register the patient below.")
        name = st.text_input("Patient Name")
        age = st.number_input("Age", min_value=0, max_value=120, step=1)
        sex = st.radio("Sex", ("Male", "Female", "Other"))
        if st.button("Register Patient"):
            new_pid = str(len(patient_db) + 1)
            patient_db[new_pid] = {"name": name, "age": age, "sex": sex, "history": []}
            st.success(f"Patient registered with ID: {new_pid}")
            st.session_state.patient_db = patient_db
            pid = new_pid
else:
    st.info("Enter a Patient ID to search for an existing record or leave blank to register a new patient.")

uploaded_file = st.file_uploader("Upload Audio File", type=["mp3", "wav", "m4a", "flac"])

if uploaded_file and pid:
    with NamedTemporaryFile(delete=False) as temp_audio:
        temp_audio.write(uploaded_file.read())
        audio_path = temp_audio.name

    st.info("Processing audio file... ⏳")
    result = model.transcribe(audio_path)
    transcription = result["text"]
    st.success("Transcription Complete! ✅")
    st.text_area("📝 Transcribed Text:", transcription, height=150)

    doc = nlp(transcription)
    medical_conditions = [ent.text for ent in doc.ents]

    st.subheader("🔬 Extracted Medical Conditions")
    st.write(medical_conditions if medical_conditions else "No conditions detected.")

    if st.button("Generate Medical Report"):
        consultation_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        prompt = f"""
        Convert the following patient consultation into a structured medical report.
        **Patient Name:** {patient_db[pid]['name']}
        **Age/Sex:** {patient_db[pid]['age']}/{patient_db[pid]['sex']}
        **Date of Consultation:** {consultation_date}
        **Transcribed Conversation:** {transcription}
        **Extracted Medical Conditions:** {', '.join(medical_conditions)}
        """
        response = llm([HumanMessage(content=prompt)])
        structured_report = response.content

        patient_db[pid]['history'].append({
            "date": consultation_date,
            "transcription": transcription,
            "medical_conditions": medical_conditions,
            "report": structured_report
        })
        st.session_state.patient_db = patient_db

        st.subheader("📄 Generated Medical Report")
        st.write(structured_report)

        report_content = "".join(
            f"\n\nDate: {entry['date']}\n{entry['report']}" for entry in patient_db[pid]['history']
        )

        def generate_pdf(content, filename):
            pdf = FPDF()
            pdf.set_auto_page_break(auto=True, margin=15)
            pdf.add_page()
            pdf.set_font("Arial", size=12)
            pdf.multi_cell(0, 10, content)
            pdf.output(filename)

        def generate_docx(content, filename):
            doc = Document()
            doc.add_paragraph(content)
            doc.save(filename)

        pdf_filename = f"Medical_Report_{pid}.pdf"
        generate_pdf(report_content, pdf_filename)
        with open(pdf_filename, "rb") as pdf_file:
            st.download_button("📥 Download PDF Report", pdf_file, pdf_filename, "application/pdf")

        docx_filename = f"Medical_Report_{pid}.docx"
        generate_docx(report_content, docx_filename)
        with open(docx_filename, "rb") as docx_file:
            st.download_button("📥 Download DOCX Report", docx_file, docx_filename, "application/vnd.openxmlformats-officedocument.wordprocessingml.document")

        st.success("Report saved and updated! ✅")




Writing app.py


In [6]:
!streamlit run app.py  &>/dev/null &

In [ ]:
!ngrok authtoken your_auth_token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
from pyngrok import ngrok

public_url = ngrok.connect(8501, "http")

print(f"Streamlit app running at: {public_url}")


Streamlit app running at: NgrokTunnel: "https://2528-34-105-19-241.ngrok-free.app" -> "http://localhost:8501"
